<a href="https://colab.research.google.com/github/anandjs11/crimedata/blob/main/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install PySpark
# http://spark.apache.org/docs/latest/api/python/index.html

!pip install pyspark==3.2

In [ ]:
# start spark sessnon and configureation
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext
spark = SparkSession.builder.master("local[2]").config("spark.driver.memory", "15g")\
.appName("Crimedata").getOrCreate()
sc = spark.sparkContext
#create an instance of SQLContext
sqlContext = SQLContext(spark)

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
# Import libraries and other functions
from io import StringIO
from collections import namedtuple
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.functions import *

import csv
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

In [ ]:
# import zipfile

# # Replace 'your_zip_file.zip' with the actual name of your uploaded zip file
# zip_file_path = '/content/gdrive/MyDrive/dataset/studenthubcrimes.zip'
# output_dir = '/content/gdrive/MyDrive/dataset/extracted'

# # Create the output directory if it doesn't exist
# !mkdir -p "extracted"

# # Extract the contents of the zip file
# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     zip_ref.extractall(output_dir)

# print("File extraction complete.")


In [ ]:
# Define the base directory path where all CSV files are located (ukcrime data)
base_directory_path = "/content/gdrive/MyDrive/dataset/extracted"

# Define the pattern to match all CSV files inside subfolders
csv_files_pattern = base_directory_path + "/*/*street.csv"

# Read all CSV files using the specified pattern
CrimeData = spark.read.option("header", "true") \
    .option("delimiter", ",") \
    .option("inferSchema", "true") \
    .csv(csv_files_pattern)

In [ ]:
CrimeData.take(2)

In [ ]:
CrimeData.show(5)

In [ ]:
CrimeData.printSchema()

In [ ]:
total_observations_crime = CrimeData.count()
print("Total observations in CrimeData:", total_observations_crime)

In [ ]:
missing_values_crime = CrimeData.select\
 ([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in CrimeData.columns])

In [ ]:
print("Missing values in CrimeData:")
missing_values_crime.show()

In [ ]:
missing_percentage_crime = missing_values_crime.select(
    [(col(c) / total_observations_crime * 100).alias(c) for c in missing_values_crime.columns]
)
print("Missing percentage in CrimeData:")
missing_percentage_crime.show()

In [ ]:
from pyspark.sql.functions import col

# Filter out rows with missing Latitude
CrimeData = CrimeData.filter(col("Latitude").isNotNull())



In [ ]:
from pyspark.sql.functions import col, sum

# Count missing values in each column
missing_values_count = CrimeData.select(*[sum(col(c).isNull().cast("int")).alias(c)\
                                          for c in CrimeData.columns])

# Convert the result to a dictionary for easier display
missing_values_dict = missing_values_count.first().asDict()

# Display missing values count for each column
for column, count in missing_values_dict.items():
    print(f"Column '{column}': {count} missing values")


In [ ]:
CrimeData = CrimeData.drop('Crime ID','Falls within','LSOA code','LSOA name',\
                           'Last outcome category','Context')
print(f"{len(CrimeData.columns)} columns in the output dataframe")

In [ ]:
CrimeData.show()

In [ ]:
from pyspark.sql.functions import col, sum

# Count missing values in each column
missing_values_count = CrimeData.select(*[sum(col(c).isNull().cast("int")).alias(c) for c in CrimeData.columns])

# Convert the result to a dictionary for easier display
missing_values_dict = missing_values_count.first().asDict()

# Display missing values count for each column
for column, count in missing_values_dict.items():
    print(f"Column '{column}': {count} missing values")

In [ ]:
# tidy up the column names

CrimeData = CrimeData.withColumnRenamed('Reported by', 'Reported_by')
CrimeData = CrimeData.withColumnRenamed('Crime type', 'Crime_type')

In [ ]:
to_csv=CrimeData
output_path = "CrimeData.csv"
to_csv.write.csv(output_path, header=True)

print("DataFrame saved to CSV file:", output_path)

In [ ]:
CrimeData.createTempView("CatData")

In [ ]:
CrimeCat = spark.sql("select distinct Crime_type from CatData")

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize Spark session
spark = SparkSession.builder.appName("CrimeAnalysis").getOrCreate()

# Load the dataset
df = CrimeData
# Count the number of rows with exact location "On or near"
count_on_or_near = df.filter(col("Location") == "On or near").count()

# Print the count
print("Count of rows with exact location 'On or near':", count_on_or_near)

# Stop the Spark session
spark.stop()


In [ ]:
CrimeCat.toPandas()

In [ ]:
BristolCrime=spark.sql("select * from CatData where Reported_by='Avon and Somerset Constabulary'")

In [ ]:
LondonCrime=spark.sql("select * from CatData where Reported_by='Metropolitan Police Service'")

In [ ]:
CambridgeCrime=spark.sql("select * from CatData where Reported_by='Cambridgeshire Constabulary'")

In [ ]:
OxfordCrime=spark.sql("select * from CatData where Reported_by='Thames Valley Police'")

In [ ]:
LeedsCrime=spark.sql("select * from CatData where Reported_by='West Yorkshire Police'")

In [ ]:
LondonCrime.toPandas()

In [ ]:
BristolCrime.toPandas()

In [ ]:
import pandas as pd
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder.appName("CrimeDataAnalysis").getOrCreate()

# Assuming 'CrimeCat' is your temporary view with crime data
# If not, replace 'CrimeCat' with your actual temporary view name
# CrimeCat = spark.table("CrimeCat")

# Convert the Spark DataFrame to a Pandas DataFrame
# pandas_df = CrimeData.toPandas()

# Extract year from the date column and add it as a new column
pandas_df['Year'] = pd.to_datetime(pandas_df['Month']).dt.year

# Group by Year, Reported_by (area), and Crime_type and calculate the count of crimes
crime_type_area_year_count = pandas_df.groupby(['Year', 'Reported_by', 'Crime_type']).size().reset_index(name='CrimeCount')

# Pivot the data to have crime types as columns
pivot_df = crime_type_area_year_count.pivot_table(index=['Year', 'Reported_by'], columns='Crime_type', values='CrimeCount', fill_value=0)

# Display the data as a table
print(pivot_df)

# Stop the Spark session
spark.stop()


In [ ]:
import pandas as pd
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder.appName("CrimeDataAnalysis").getOrCreate()

# Assuming 'CrimeCat' is your temporary view with crime data
# If not, replace 'CrimeCat' with your actual temporary view name
# CrimeCat = spark.table("CrimeCat")

# Convert the Spark DataFrame to a Pandas DataFrame
pandas_df = CrimeData.toPandas()

# Group by month, city, and crime type and calculate the count of crimes
crime_type_month_city_count = pandas_df.groupby(['Month', 'Reported_by', 'Crime_type']).size().reset_index(name='CrimeCount')

# Pivot the data to have crime types as columns
pivot_df = crime_type_month_city_count.pivot_table(index=['Month', 'Reported_by'], columns='Crime_type', values='CrimeCount', fill_value=0)

# Display the data as a table
print(pivot_df)

# Stop the Spark session
spark.stop()


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year

# Initialize Spark session
spark = SparkSession.builder.appName("CrimeAnalysis").getOrCreate()

# Load the dataset
# CrimeData = spark.read.option("header", "true").csv("path_to_your_dataset.csv")
df=CrimeData
# Filter the data for the year 2021 and "Metropolitan Police Service"
filtered_data = df.filter((year(col("Month")) == 2022) & (col("Reported_by") == "Metropolitan Police Service"))

# Group by "Reported by" and "Month" and count the number of crimes
crime_counts = filtered_data.groupBy("Reported_by", "Month").count()

# Show the resulting counts
crime_counts.orderBy("count").show(12)


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year

# Initialize Spark session
spark = SparkSession.builder.appName("CrimeAnalysis").getOrCreate()

# Load the dataset
# CrimeData = spark.read.option("header", "true").csv("path_to_your_dataset.csv")
df=CrimeData
# Filter the data for the year 2021 and "Metropolitan Police Service"
filtered_data = df.filter((year(col("Month")) == 2022) & (col("Reported_by") == "Cambridgeshire Constabulary"))

# Group by "Reported by" and "Month" and count the number of crimes
crime_counts = filtered_data.groupBy("Reported_by", "Month").count()

# Show the resulting counts
crime_counts.orderBy("count").show(12)


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year

# Initialize Spark session
spark = SparkSession.builder.appName("CrimeAnalysis").getOrCreate()

# Load the dataset
# CrimeData = spark.read.option("header", "true").csv("path_to_your_dataset.csv")
df=CrimeData
# Filter the data for the year 2021 and "Metropolitan Police Service"
filtered_data = df.filter((year(col("Month")) == 2022) & (col("Reported_by") == "Avon and Somerset Constabulary"))

# Group by "Reported by" and "Month" and count the number of crimes
crime_counts = filtered_data.groupBy("Reported_by", "Month").count()

# Show the resulting counts
crime_counts.orderBy("count").show(12)


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year

# Initialize Spark session
spark = SparkSession.builder.appName("CrimeAnalysis").getOrCreate()

# Load the dataset
# CrimeData = spark.read.option("header", "true").csv("path_to_your_dataset.csv")
df=CrimeData
# Filter the data for the year 2021 and "Metropolitan Police Service"
filtered_data = df.filter((year(col("Month")) == 2022) & (col("Reported_by") == "Thames Valley Police"))

# Group by "Reported by" and "Month" and count the number of crimes
crime_counts = filtered_data.groupBy("Reported_by", "Month").count()

# Show the resulting counts
crime_counts.orderBy("count").show(12)


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year

# Initialize Spark session
spark = SparkSession.builder.appName("CrimeAnalysis").getOrCreate()

# Load the dataset
# CrimeData = spark.read.option("header", "true").csv("path_to_your_dataset.csv")
df=CrimeData
# Filter the data for the year 2021 and "Metropolitan Police Service"
filtered_data = df.filter((year(col("Month")) == 2022) & (col("Reported_by") == "West Yorkshire Police"))

# Group by "Reported by" and "Month" and count the number of crimes
crime_counts = filtered_data.groupBy("Reported_by", "Month").count()

# Show the resulting counts
crime_counts.orderBy("count").show(12)


In [ ]:
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder.appName("CrimeDataAnalysis").getOrCreate()

# Load the dataset
# CrimeData = spark.read.option("header", "true").csv("path_to_your_dataset.csv")

# Assuming 'CrimeData' is your DataFrame
# If not, replace 'CrimeData' with your actual DataFrame name

# Convert the Spark DataFrame to a Pandas DataFrame
# pandas_df = CrimeData.toPandas()

# Filter the data for the years 2021 and 2022
filtered_data = pandas_df[pandas_df['Month'].str.startswith(('2021', '2022'))]

# Group by month, reported by, and crime type and calculate the count of crimes
crime_type_month_city_count = filtered_data.groupby(['Month', 'Reported_by', 'Crime_type']).size().reset_index(name='CrimeCount')

# Pivot the data to have crime types as columns
pivot_df = crime_type_month_city_count.pivot_table(index=['Month', 'Reported_by'], columns='Crime_type', values='CrimeCount', fill_value=0)

# Plot the data
for city in pivot_df.index.get_level_values('Reported_by').unique():
    city_data = pivot_df.loc[pivot_df.index.get_level_values('Reported_by') == city]

    plt.figure(figsize=(20, 8))
    city_data.plot(kind='bar', ax=plt.gca())

    plt.title(f'Crime Types Distribution Each Month in {city}')
    plt.xlabel('Month')
    plt.ylabel('Crime Count')
    plt.legend(loc='upper left')
    plt.xticks(rotation=45, ha="right")  # Rotating x-axis labels for readability

    plt.show()

# Stop the Spark session
spark.stop()


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year

# Initialize Spark session
spark = SparkSession.builder.appName("CrimeAnalysis").getOrCreate()

# Load the dataset
df=CrimeData
# Filter the data for the year 2021 and "Metropolitan Police Service"
data_2021 = df.filter(
    (year(col("Month")) == 2021) &
    (col("Reported by") == "West Yorkshire Police")
)

# Filter the data for the year 2022 and "Metropolitan Police Service"
data_2022 = df.filter(
    (year(col("Month")) == 2022) &
    (col("Reported by") == "West Yorkshire Police")
)

# Group by "Crime type" and count the occurrences for 2021
crime_type_counts_2021 = data_2021.groupBy("Crime_type").count()

# Group by "Crime type" and count the occurrences for 2022
crime_type_counts_2022 = data_2022.groupBy("Crime_type").count()

# Show the resulting counts for 2021
print("Crime Type Counts for 2021:")
crime_type_counts_2021.show(truncate=False)

# Show the resulting counts for 2022
print("Crime Type Counts for 2022:")
crime_type_counts_2022.show(truncate=False)


In [ ]:
import pandas as pd

input_path="/content/CrimeData.csv"
crime_data= spark.read.csv(input_path, header=True, inferSchema=True)
crime_data=crime_data.toPandas()

# Convert the "Month" column to a datetime format
crime_data['Month'] = pd.to_datetime(crime_data['Month'])

# Filter data for the year 2021 and for "Violence and sexual offences" crime type
violence_sexual_offences_2021 = crime_data[
    (crime_data['Month'].dt.year == 2021) &
    (crime_data['Crime_type'] == 'Violence and sexual offences')&
    (crime_data['Reported_by']=='Metropolitan Police Service')
]

# Count the total number of reported incidents
total_incidents_2021 = len(violence_sexual_offences_2021)

print("Total number of Violence and Sexual Offences reported in 2021:", total_incidents_2021)


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize a Spark session
spark = SparkSession.builder.appName("CrimeDataAnalysis").getOrCreate()

# Load the dataset into a DataFrame (assuming the variable 'df' contains your data)
df = CrimeData

# Filter for 'Violence and sexual offences'
violence_sexual_offences = df.filter(col("Crime_type") == "Violence and sexual offences")

# Group by 'Location' and count the occurrences
location_counts = violence_sexual_offences.groupBy("Location").count()

# Find the location with the maximum count
max_location = location_counts.orderBy(col("count").desc()).first()

print("Location with Maximum Count of Violence and Sexual Offences:")
print("Location:", max_location["Location"])
print("Count:", max_location["count"])

# Stop the Spark session
spark.stop()


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize a Spark session
spark = SparkSession.builder.appName("CrimeDataAnalysis").getOrCreate()

# Load the dataset into a DataFrame (assuming the variable 'df' contains your data)
df = CrimeData

# Filter for rows where Location is exactly "On or near"
on_or_near_locations = df.filter(col("Location") == "On or near ")

# Display the top 10 rows
top_10_on_or_near_locations = on_or_near_locations.limit(10)
top_10_on_or_near_locations.show()

# Stop the Spark session
spark.stop()


Violence and sexual offences 2021

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize a Spark session
spark = SparkSession.builder.appName("CrimeDataAnalysis").getOrCreate()

# Load data from CSV file into a Spark DataFrame (replace with your input path)

crime_data = CrimeData

# Filter relevant columns
filtered_data = crime_data.select("Reported_by", "Location", "Crime_type")

# Group by "Reported_by" and "Location", and count occurrences
location_counts = filtered_data.groupBy("Reported_by", "Location").count()

# Create a temporary view for the location counts
location_counts.createOrReplaceTempView("LocationCounts")

# Get unique "Reported by" values
reported_by_list = location_counts.select("Reported_by").distinct().collect()

# Loop through each "Reported by" and get the top 10 locations
for reported_by_row in reported_by_list:
    reported_by = reported_by_row.Reported_by

    top_10_locations = spark.sql(f"""
        SELECT Location, count
        FROM LocationCounts
        WHERE Reported_by = '{reported_by}'
        ORDER BY count DESC
    """)

    # Show the results for each "Reported by"
    print(f"Top 10 Locations for {reported_by}:")
    top_10_locations.show()

# Stop the Spark session
spark.stop()


In [ ]:
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize Spark session
spark = SparkSession.builder.appName("CrimeAnalysis").getOrCreate()

# Load the dataset
df = CrimeData

# Get unique "Reported by" values
reported_by_list = df.select("Reported_by").dropDuplicates().collect()

# Filter the data for each "Reported by" and create a pie chart
for reported_by_row in reported_by_list:
    reported_by = reported_by_row.Reported_by

    filtered_data = df.filter(
        (col("Crime type") == "Violence and sexual offences") &
        (col("Reported by") == reported_by) &
        (~col("Location").isin(["on or near ", "On or near "])))

    # Group by "Location" and count the occurrences
    location_counts = filtered_data.groupBy("Location").count()

    top_10_locations = location_counts.orderBy(col("count").desc()).limit(10)
    top_10_locations.show()

    # Convert Spark DataFrame to Pandas DataFrame
    pandas_df = top_10_locations.toPandas()

    # Create a pie chart
    plt.figure(figsize=(10, 6))
    plt.pie(pandas_df["count"], labels=pandas_df["Location"], autopct="%1.1f%%", startangle=140)
    plt.axis("equal")  # Equal aspect ratio ensures that pie is drawn as a circle.
    plt.title(f"Locations of Violence and Sexual Offences ({reported_by})")
    plt.show()

# Stop the Spark session
spark.stop()


In [ ]:
import folium
from folium.plugins import HeatMap
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize Spark session
spark = SparkSession.builder.appName("CrimeAnalysis").getOrCreate()

# Load the dataset
df = CrimeData

# Get unique "Reported by" values
reported_by_list = df.select("Reported_by").dropDuplicates().collect()

# Create a map centered at a specified location
m = folium.Map(location=[51.5074, -0.1278], zoom_start=10, tiles='cartodbpositron')

# Loop through each "Reported by" value
for reported_by_row in reported_by_list:
    reported_by = reported_by_row.Reported_by

    filtered_data = df.filter(
        (col("Crime type") == "Violence and sexual offences") &
        (col("Reported by") == reported_by) &
        (~col("Location").isin(["on or near ", "On or near "])))

    # Group by latitude and longitude and count the occurrences
    location_counts = filtered_data.groupBy("Latitude", "Longitude").count()

    top_10_locations = location_counts.orderBy(col("count").desc()).limit(10)
    # top_10_locations.show()

    # Convert Spark DataFrame to Pandas DataFrame
    pandas_df = top_10_locations.toPandas()

    # Convert location data to a list of tuples with latitude, longitude, and count
    heat_data = [(row['Latitude'], row['Longitude'], 1) for index, row in pandas_df.iterrows()]

    # Create a HeatMap layer
    HeatMap(data=heat_data, radius=15).add_to(m)

# Display the map
m.save("heatmap.html")

# Stop the Spark session
spark.stop()


In [ ]:
m.save("heatmap.html")

In [ ]:
import folium
from folium.plugins import HeatMap
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize a Spark session
spark = SparkSession.builder.appName("CrimeDataAnalysis").getOrCreate()

# Filter relevant columns (Longitude, Latitude, Crime_type, Location, Month)
location_data = df.select("Longitude", "Latitude", "Crime_type", "Location", "Month")

# Convert the "Month" column to a date format
location_data = location_data.withColumn("Month", col("Month").cast("date"))

# Filter only 'Violence and sexual offences' incidents in the year 2021
violence_sexual_offences_2021 = location_data.filter(
    (col("Crime_type") == "Violence and sexual offences") &
    (col("Month").between("2021-01", "2021-12"))
)

# Create a map centered at the specified location
m = folium.Map(location=[51.5074, -0.1278], zoom_start=10, tiles='cartodbpositron')

# Convert location data to a list of tuples with latitude, longitude, and count
heat_data = [(row.Latitude, row.Longitude, 1) for row in violence_sexual_offences_2021.collect()]

# Create a HeatMap layer
HeatMap(data=heat_data, radius=15).add_to(m)

# Display the map
m


In [ ]:
import folium
from folium.plugins import HeatMap
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder.appName("CrimeDataAnalysis").getOrCreate()

# Filter relevant columns (Longitude, Latitude, Crime_type, Month)
location_data = pandas_df[['Longitude', 'Latitude', 'Crime_type', 'Month']]

# Convert the "Month" column to a datetime format
location_data['Month'] = pd.to_datetime(location_data['Month'])

# Filter only 'Violence and sexual offences' incidents in the year 2021
violence_sexual_offences_2021 = location_data[
    (location_data['Crime_type'] == 'Violence and sexual offences') &
    (location_data['Month'].dt.year == 2021)
]

# Create a map centered at the specified location
m = folium.Map(location=[51.4203, 0.0705], zoom_start=12, tiles='cartodbpositron')

# Convert location data to a list of tuples with latitude, longitude, and count
heat_data = [(row['Latitude'], row['Longitude'], 1) for index, row in violence_sexual_offences_2021.iterrows()]

# Create a HeatMap layer
HeatMap(data=heat_data, radius=15).add_to(m)

# Display the map
m


In [ ]:
import folium
from folium.plugins import HeatMap
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder.appName("CrimeDataAnalysis").getOrCreate()

# Filter relevant columns (Longitude, Latitude, Crime_type, Month)
location_data = pandas_df[['Longitude', 'Latitude', 'Crime_type', 'Month']]

# Convert the "Month" column to a datetime format
location_data['Month'] = pd.to_datetime(location_data['Month'])

# Filter only 'Violence and sexual offences' incidents in the year 2021
violence_sexual_offences_2021 = location_data[
    (location_data['Crime_type'] == 'Violence and sexual offences') &
    (location_data['Month'].dt.year == 2023)
]

# Create a map centered at the specified location
m = folium.Map(location=[51.4203, 0.0705], zoom_start=12, tiles='cartodbpositron')

# Convert location data to a list of tuples with latitude, longitude, and count
heat_data = [(row['Latitude'], row['Longitude'], 1) for index, row in violence_sexual_offences_2021.iterrows()]

# Create a HeatMap layer
HeatMap(data=heat_data, radius=15).add_to(m)

# Display the map
m


In [ ]:
import folium
from folium.plugins import HeatMap
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder.appName("CrimeDataAnalysis").getOrCreate()

# Filter relevant columns (Longitude, Latitude, Crime_type, Month)
location_data = pandas_df[['Longitude', 'Latitude', 'Crime_type', 'Month']]

# Convert the "Month" column to a datetime format
location_data['Month'] = pd.to_datetime(location_data['Month'])

# Filter only 'Violence and sexual offences' incidents in the year 2021
violence_sexual_offences_2021 = location_data[
    (location_data['Crime_type'] == 'Bicycle theft') &
    (location_data['Month'].dt.year == 2021)
]

# Create a map centered at the specified location
m = folium.Map(location=[51.4203, 0.0705], zoom_start=12, tiles='cartodbpositron')

# Convert location data to a list of tuples with latitude, longitude, and count
heat_data = [(row['Latitude'], row['Longitude'], 1) for index, row in violence_sexual_offences_2021.iterrows()]

# Create a HeatMap layer
HeatMap(data=heat_data, radius=15).add_to(m)

# Display the map
m


In [ ]:
import folium
from folium.plugins import HeatMap
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder.appName("CrimeDataAnalysis").getOrCreate()

# Filter relevant columns (Longitude, Latitude, Crime_type, Month)
location_data = pandas_df[['Longitude', 'Latitude', 'Crime_type', 'Month']]

# Convert the "Month" column to a datetime format
location_data['Month'] = pd.to_datetime(location_data['Month'])

# Filter only 'Violence and sexual offences' incidents in the year 2021
violence_sexual_offences_2021 = location_data[
    (location_data['Crime_type'] == 'Bicycle theft') &
    (location_data['Month'].dt.year == 2022)
]

# Create a map centered at the specified location
m = folium.Map(location=[51.4203, 0.0705], zoom_start=12, tiles='cartodbpositron')

# Convert location data to a list of tuples with latitude, longitude, and count
heat_data = [(row['Latitude'], row['Longitude'], 1) for index, row in violence_sexual_offences_2021.iterrows()]

# Create a HeatMap layer
HeatMap(data=heat_data, radius=15).add_to(m)

# Display the map
m


In [ ]:
import folium
from folium.plugins import HeatMap
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder.appName("CrimeDataAnalysis").getOrCreate()

# Filter relevant columns (Longitude, Latitude, Crime_type, Month)
location_data = pandas_df[['Longitude', 'Latitude', 'Crime_type', 'Month']]

# Convert the "Month" column to a datetime format
location_data['Month'] = pd.to_datetime(location_data['Month'])

# Filter only 'Violence and sexual offences' incidents in the year 2021
violence_sexual_offences_2021 = location_data[
    (location_data['Crime_type'] == 'Bicycle theft') &
    (location_data['Month'].dt.year == 2023)
]

# Create a map centered at the specified location
m = folium.Map(location=[51.4203, 0.0705], zoom_start=12, tiles='cartodbpositron')

# Convert location data to a list of tuples with latitude, longitude, and count
heat_data = [(row['Latitude'], row['Longitude'], 1) for index, row in violence_sexual_offences_2021.iterrows()]

# Create a HeatMap layer
HeatMap(data=heat_data, radius=15).add_to(m)

# Display the map
m


In [ ]:
import folium
from folium.plugins import HeatMap
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder.appName("CrimeDataAnalysis").getOrCreate()

# Filter relevant columns (Longitude, Latitude, Crime_type, Month)
location_data = pandas_df[['Longitude', 'Latitude', 'Crime_type', 'Month']]

# Convert the "Month" column to a datetime format
location_data['Month'] = pd.to_datetime(location_data['Month'])

# Filter only 'Violence and sexual offences' incidents in the year 2021
violence_sexual_offences_2021 = location_data[
    (location_data['Crime_type'] == 'Public order') &
    (location_data['Month'].dt.year == 2021)
]

# Create a map centered at the specified location
m = folium.Map(location=[51.4203, 0.0705], zoom_start=12, tiles='cartodbpositron')

# Convert location data to a list of tuples with latitude, longitude, and count
heat_data = [(row['Latitude'], row['Longitude'], 1) for index, row in violence_sexual_offences_2021.iterrows()]

# Create a HeatMap layer
HeatMap(data=heat_data, radius=15).add_to(m)

# Display the map
m


In [ ]:
import folium
from folium.plugins import HeatMap
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder.appName("CrimeDataAnalysis").getOrCreate()

# Filter relevant columns (Longitude, Latitude, Crime_type, Month)
location_data = pandas_df[['Longitude', 'Latitude', 'Crime_type', 'Month']]

# Convert the "Month" column to a datetime format
location_data['Month'] = pd.to_datetime(location_data['Month'])

# Filter only 'Violence and sexual offences' incidents in the year 2021
violence_sexual_offences_2021 = location_data[
    (location_data['Crime_type'] == 'Public order') &
    (location_data['Month'].dt.year == 2022)
]

# Create a map centered at the specified location
m = folium.Map(location=[51.4203, 0.0705], zoom_start=12, tiles='cartodbpositron')

# Convert location data to a list of tuples with latitude, longitude, and count
heat_data = [(row['Latitude'], row['Longitude'], 1) for index, row in violence_sexual_offences_2021.iterrows()]

# Create a HeatMap layer
HeatMap(data=heat_data, radius=15).add_to(m)

# Display the map
m


In [ ]:
import folium
from folium.plugins import HeatMap
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder.appName("CrimeDataAnalysis").getOrCreate()

# Filter relevant columns (Longitude, Latitude, Crime_type, Month)
location_data = pandas_df[['Longitude', 'Latitude', 'Crime_type', 'Month']]

# Convert the "Month" column to a datetime format
location_data['Month'] = pd.to_datetime(location_data['Month'])

# Filter only 'Violence and sexual offences' incidents in the year 2021
violence_sexual_offences_2021 = location_data[
    (location_data['Crime_type'] == 'Public order') &
    (location_data['Month'].dt.year == 2023)
]

# Create a map centered at the specified location
m = folium.Map(location=[51.4203, 0.0705], zoom_start=12, tiles='cartodbpositron')

# Convert location data to a list of tuples with latitude, longitude, and count
heat_data = [(row['Latitude'], row['Longitude'], 1) for index, row in violence_sexual_offences_2021.iterrows()]

# Create a HeatMap layer
HeatMap(data=heat_data, radius=15).add_to(m)

# Display the map
m


In [ ]:
import folium
from folium.plugins import HeatMap
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder.appName("CrimeDataAnalysis").getOrCreate()

# Filter relevant columns (Longitude, Latitude, Crime_type, Month)
location_data = pandas_df[['Longitude', 'Latitude', 'Crime_type', 'Month']]

# Convert the "Month" column to a datetime format
location_data['Month'] = pd.to_datetime(location_data['Month'])

# Filter only 'Violence and sexual offences' incidents in the year 2021
violence_sexual_offences_2021 = location_data[
    (location_data['Crime_type'] == 'Drugs') &
    (location_data['Month'].dt.year == 2021)
]

# Create a map centered at the specified location
m = folium.Map(location=[51.4203, 0.0705], zoom_start=12, tiles='cartodbpositron')

# Convert location data to a list of tuples with latitude, longitude, and count
heat_data = [(row['Latitude'], row['Longitude'], 1) for index, row in violence_sexual_offences_2021.iterrows()]

# Create a HeatMap layer
HeatMap(data=heat_data, radius=15).add_to(m)

# Display the map
m


In [ ]:
import folium
from folium.plugins import HeatMap
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder.appName("CrimeDataAnalysis").getOrCreate()

# Filter relevant columns (Longitude, Latitude, Crime_type, Month)
location_data = pandas_df[['Longitude', 'Latitude', 'Crime_type', 'Month']]

# Convert the "Month" column to a datetime format
location_data['Month'] = pd.to_datetime(location_data['Month'])

# Filter only 'Violence and sexual offences' incidents in the year 2021
violence_sexual_offences_2021 = location_data[
    (location_data['Crime_type'] == 'Drugs') &
    (location_data['Month'].dt.year == 2022)
]

# Create a map centered at the specified location
m = folium.Map(location=[51.4203, 0.0705], zoom_start=12, tiles='cartodbpositron')

# Convert location data to a list of tuples with latitude, longitude, and count
heat_data = [(row['Latitude'], row['Longitude'], 1) for index, row in violence_sexual_offences_2021.iterrows()]

# Create a HeatMap layer
HeatMap(data=heat_data, radius=15).add_to(m)

# Display the map
m


In [ ]:
import folium
from folium.plugins import HeatMap
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder.appName("CrimeDataAnalysis").getOrCreate()

# Filter relevant columns (Longitude, Latitude, Crime_type, Month)
location_data = pandas_df[['Longitude', 'Latitude', 'Crime_type', 'Month']]

# Convert the "Month" column to a datetime format
location_data['Month'] = pd.to_datetime(location_data['Month'])

# Filter only 'Violence and sexual offences' incidents in the year 2021
violence_sexual_offences_2021 = location_data[
    (location_data['Crime_type'] == 'Drugs') &
    (location_data['Month'].dt.year == 2023)
]

# Create a map centered at the specified location
m = folium.Map(location=[51.4203, 0.0705], zoom_start=12, tiles='cartodbpositron')

# Convert location data to a list of tuples with latitude, longitude, and count
heat_data = [(row['Latitude'], row['Longitude'], 1) for index, row in violence_sexual_offences_2021.iterrows()]

# Create a HeatMap layer
HeatMap(data=heat_data, radius=15).add_to(m)

# Display the map
m


In [ ]:
import folium
from folium.plugins import HeatMap
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder.appName("CrimeDataAnalysis").getOrCreate()

# Filter relevant columns (Longitude, Latitude, Crime_type, Month)
location_data = pandas_df[['Longitude', 'Latitude', 'Crime_type', 'Month']]

# Convert the "Month" column to a datetime format
location_data['Month'] = pd.to_datetime(location_data['Month'])

# Filter only 'Violence and sexual offences' incidents in the year 2021
violence_sexual_offences_2021 = location_data[
    (location_data['Crime_type'] == 'Other crime') &
    (location_data['Month'].dt.year == 2021)
]

# Create a map centered at the specified location
m = folium.Map(location=[51.4203, 0.0705], zoom_start=12, tiles='cartodbpositron')

# Convert location data to a list of tuples with latitude, longitude, and count
heat_data = [(row['Latitude'], row['Longitude'], 1) for index, row in violence_sexual_offences_2021.iterrows()]

# Create a HeatMap layer
HeatMap(data=heat_data, radius=15).add_to(m)

# Display the map
m


In [ ]:
import folium
from folium.plugins import HeatMap
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder.appName("CrimeDataAnalysis").getOrCreate()

# Filter relevant columns (Longitude, Latitude, Crime_type, Month)
location_data = pandas_df[['Longitude', 'Latitude', 'Crime_type', 'Month']]

# Convert the "Month" column to a datetime format
location_data['Month'] = pd.to_datetime(location_data['Month'])

# Filter only 'Violence and sexual offences' incidents in the year 2021
violence_sexual_offences_2021 = location_data[
    (location_data['Crime_type'] == 'Other crime') &
    (location_data['Month'].dt.year == 2022)
]

# Create a map centered at the specified location
m = folium.Map(location=[51.4203, 0.0705], zoom_start=12, tiles='cartodbpositron')

# Convert location data to a list of tuples with latitude, longitude, and count
heat_data = [(row['Latitude'], row['Longitude'], 1) for index, row in violence_sexual_offences_2021.iterrows()]

# Create a HeatMap layer
HeatMap(data=heat_data, radius=15).add_to(m)

# Display the map
m


In [ ]:
import folium
from folium.plugins import HeatMap
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder.appName("CrimeDataAnalysis").getOrCreate()

# Filter relevant columns (Longitude, Latitude, Crime_type, Month)
location_data = pandas_df[['Longitude', 'Latitude', 'Crime_type', 'Month']]

# Convert the "Month" column to a datetime format
location_data['Month'] = pd.to_datetime(location_data['Month'])

# Filter only 'Violence and sexual offences' incidents in the year 2021
violence_sexual_offences_2021 = location_data[
    (location_data['Crime_type'] == 'Other crime') &
    (location_data['Month'].dt.year == 2023)
]

# Create a map centered at the specified location
m = folium.Map(location=[51.4203, 0.0705], zoom_start=12, tiles='cartodbpositron')

# Convert location data to a list of tuples with latitude, longitude, and count
heat_data = [(row['Latitude'], row['Longitude'], 1) for index, row in violence_sexual_offences_2021.iterrows()]

# Create a HeatMap layer
HeatMap(data=heat_data, radius=15).add_to(m)

# Display the map
m


In [ ]:
import folium
from folium.plugins import HeatMap
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder.appName("CrimeDataAnalysis").getOrCreate()

# Filter relevant columns (Longitude, Latitude, Crime_type, Month)
location_data = pandas_df[['Longitude', 'Latitude', 'Crime_type', 'Month']]

# Convert the "Month" column to a datetime format
location_data['Month'] = pd.to_datetime(location_data['Month'])

# Filter only 'Violence and sexual offences' incidents in the year 2021
violence_sexual_offences_2021 = location_data[
    (location_data['Crime_type'] == 'Robbery') &
    (location_data['Month'].dt.year == 2021)
]

# Create a map centered at the specified location
m = folium.Map(location=[51.4203, 0.0705], zoom_start=12, tiles='cartodbpositron')

# Convert location data to a list of tuples with latitude, longitude, and count
heat_data = [(row['Latitude'], row['Longitude'], 1) for index, row in violence_sexual_offences_2021.iterrows()]

# Create a HeatMap layer
HeatMap(data=heat_data, radius=15).add_to(m)

# Display the map
m


In [ ]:
import folium
from folium.plugins import HeatMap
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder.appName("CrimeDataAnalysis").getOrCreate()

# Filter relevant columns (Longitude, Latitude, Crime_type, Month)
location_data = pandas_df[['Longitude', 'Latitude', 'Crime_type', 'Month']]

# Convert the "Month" column to a datetime format
location_data['Month'] = pd.to_datetime(location_data['Month'])

# Filter only 'Violence and sexual offences' incidents in the year 2021
violence_sexual_offences_2021 = location_data[
    (location_data['Crime_type'] == 'Robbery') &
    (location_data['Month'].dt.year == 2022)
]

# Create a map centered at the specified location
m = folium.Map(location=[51.4203, 0.0705], zoom_start=12, tiles='cartodbpositron')

# Convert location data to a list of tuples with latitude, longitude, and count
heat_data = [(row['Latitude'], row['Longitude'], 1) for index, row in violence_sexual_offences_2021.iterrows()]

# Create a HeatMap layer
HeatMap(data=heat_data, radius=15).add_to(m)

# Display the map
m


In [ ]:
import folium
from folium.plugins import HeatMap
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder.appName("CrimeDataAnalysis").getOrCreate()

# Filter relevant columns (Longitude, Latitude, Crime_type, Month)
location_data = pandas_df[['Longitude', 'Latitude', 'Crime_type', 'Month']]

# Convert the "Month" column to a datetime format
location_data['Month'] = pd.to_datetime(location_data['Month'])

# Filter only 'Violence and sexual offences' incidents in the year 2021
violence_sexual_offences_2021 = location_data[
    (location_data['Crime_type'] == 'Robbery') &
    (location_data['Month'].dt.year == 2023)
]

# Create a map centered at the specified location
m = folium.Map(location=[51.4203, 0.0705], zoom_start=12, tiles='cartodbpositron')

# Convert location data to a list of tuples with latitude, longitude, and count
heat_data = [(row['Latitude'], row['Longitude'], 1) for index, row in violence_sexual_offences_2021.iterrows()]

# Create a HeatMap layer
HeatMap(data=heat_data, radius=15).add_to(m)

# Display the map
m


In [ ]:
import folium
from folium.plugins import HeatMap
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder.appName("CrimeDataAnalysis").getOrCreate()

# Filter relevant columns (Longitude, Latitude, Crime_type, Month)
location_data = pandas_df[['Longitude', 'Latitude', 'Crime_type', 'Month']]

# Convert the "Month" column to a datetime format
location_data['Month'] = pd.to_datetime(location_data['Month'])

# Filter only 'Violence and sexual offences' incidents in the year 2021
violence_sexual_offences_2021 = location_data[
    (location_data['Crime_type'] == 'Criminal damage and arson') &
    (location_data['Month'].dt.year == 2021)
]

# Create a map centered at the specified location
m = folium.Map(location=[51.4203, 0.0705], zoom_start=12, tiles='cartodbpositron')

# Convert location data to a list of tuples with latitude, longitude, and count
heat_data = [(row['Latitude'], row['Longitude'], 1) for index, row in violence_sexual_offences_2021.iterrows()]

# Create a HeatMap layer
HeatMap(data=heat_data, radius=15).add_to(m)

# Display the map
m


In [ ]:
import folium
from folium.plugins import HeatMap
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder.appName("CrimeDataAnalysis").getOrCreate()

# Filter relevant columns (Longitude, Latitude, Crime_type, Month)
location_data = pandas_df[['Longitude', 'Latitude', 'Crime_type', 'Month']]

# Convert the "Month" column to a datetime format
location_data['Month'] = pd.to_datetime(location_data['Month'])

# Filter only 'Violence and sexual offences' incidents in the year 2021
violence_sexual_offences_2021 = location_data[
    (location_data['Crime_type'] == 'Criminal damage and arson') &
    (location_data['Month'].dt.year == 2022)
]

# Create a map centered at the specified location
m = folium.Map(location=[51.4203, 0.0705], zoom_start=12, tiles='cartodbpositron')

# Convert location data to a list of tuples with latitude, longitude, and count
heat_data = [(row['Latitude'], row['Longitude'], 1) for index, row in violence_sexual_offences_2021.iterrows()]

# Create a HeatMap layer
HeatMap(data=heat_data, radius=15).add_to(m)

# Display the map
m


In [ ]:
import folium
from folium.plugins import HeatMap
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder.appName("CrimeDataAnalysis").getOrCreate()

# Filter relevant columns (Longitude, Latitude, Crime_type, Month)
location_data = pandas_df[['Longitude', 'Latitude', 'Crime_type', 'Month']]

# Convert the "Month" column to a datetime format
location_data['Month'] = pd.to_datetime(location_data['Month'])

# Filter only 'Violence and sexual offences' incidents in the year 2021
violence_sexual_offences_2021 = location_data[
    (location_data['Crime_type'] == 'Criminal damage and arson') &
    (location_data['Month'].dt.year == 2023)
]

# Create a map centered at the specified location
m = folium.Map(location=[51.4203, 0.0705], zoom_start=12, tiles='cartodbpositron')

# Convert location data to a list of tuples with latitude, longitude, and count
heat_data = [(row['Latitude'], row['Longitude'], 1) for index, row in violence_sexual_offences_2021.iterrows()]

# Create a HeatMap layer
HeatMap(data=heat_data, radius=15).add_to(m)

# Display the map
m


In [ ]:
import folium
from folium.plugins import HeatMap
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder.appName("CrimeDataAnalysis").getOrCreate()

# Filter relevant columns (Longitude, Latitude, Crime_type, Month)
location_data = pandas_df[['Longitude', 'Latitude', 'Crime_type', 'Month']]

# Convert the "Month" column to a datetime format
location_data['Month'] = pd.to_datetime(location_data['Month'])

# Filter only 'Violence and sexual offences' incidents in the year 2021
violence_sexual_offences_2021 = location_data[
    (location_data['Crime_type'] == 'Theft from the person') &
    (location_data['Month'].dt.year == 2021)
]

# Create a map centered at the specified location
m = folium.Map(location=[51.4203, 0.0705], zoom_start=12, tiles='cartodbpositron')

# Convert location data to a list of tuples with latitude, longitude, and count
heat_data = [(row['Latitude'], row['Longitude'], 1) for index, row in violence_sexual_offences_2021.iterrows()]

# Create a HeatMap layer
HeatMap(data=heat_data, radius=15).add_to(m)

# Display the map
m


In [ ]:
import folium
from folium.plugins import HeatMap
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder.appName("CrimeDataAnalysis").getOrCreate()

# Filter relevant columns (Longitude, Latitude, Crime_type, Month)
location_data = pandas_df[['Longitude', 'Latitude', 'Crime_type', 'Month']]

# Convert the "Month" column to a datetime format
location_data['Month'] = pd.to_datetime(location_data['Month'])

# Filter only 'Violence and sexual offences' incidents in the year 2021
violence_sexual_offences_2021 = location_data[
    (location_data['Crime_type'] == 'Theft from the person') &
    (location_data['Month'].dt.year == 2022)
]

# Create a map centered at the specified location
m = folium.Map(location=[51.4203, 0.0705], zoom_start=12, tiles='cartodbpositron')

# Convert location data to a list of tuples with latitude, longitude, and count
heat_data = [(row['Latitude'], row['Longitude'], 1) for index, row in violence_sexual_offences_2021.iterrows()]

# Create a HeatMap layer
HeatMap(data=heat_data, radius=15).add_to(m)

# Display the map
m


In [ ]:
import folium
from folium.plugins import HeatMap
from pyspark.sql import SparkSession

# Initialize a Spark session
spark = SparkSession.builder.appName("CrimeDataAnalysis").getOrCreate()

# Filter relevant columns (Longitude, Latitude, Crime_type, Month)
location_data = pandas_df[['Longitude', 'Latitude', 'Crime_type', 'Month']]

# Convert the "Month" column to a datetime format
location_data['Month'] = pd.to_datetime(location_data['Month'])

# Filter only 'Violence and sexual offences' incidents in the year 2021
violence_sexual_offences_2021 = location_data[
    (location_data['Crime_type'] == 'Theft from the person') &
    (location_data['Month'].dt.year == 2023)
]

# Create a map centered at the specified location
m = folium.Map(location=[51.4203, 0.0705], zoom_start=12, tiles='cartodbpositron')

# Convert location data to a list of tuples with latitude, longitude, and count
heat_data = [(row['Latitude'], row['Longitude'], 1) for index, row in violence_sexual_offences_2021.iterrows()]

# Create a HeatMap layer
HeatMap(data=heat_data, radius=15).add_to(m)

# Display the map
m
